3 Modals interacting with each other. We'll use OpenAI, Claude and gemini modals in this.

For OpenAI, visit https://openai.com/api/
For Anthropic, visit https://console.anthropic.com/
For Google, visit https://ai.google.dev/gemini-api

In [1]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
from IPython.display import Markdown, display, update_display

In [2]:
# Load environment variables in a file called .env

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY', 'your-key-if-not-using-env')
os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY', 'your-key-if-not-using-env')

In [3]:
# Connect to OpenAI, Anthropic and Google
# All 3 APIs are similar
# Having problems with API files? You can use openai = OpenAI(api_key="your-key-here") and same for claude
# Having problems with Google Gemini setup? Then just skip Gemini; you'll get all the experience you need from GPT and Claude.

openai = OpenAI()

claude = anthropic.Anthropic()

google.generativeai.configure()

In [46]:
# Let's make a conversation between GPT-4o-mini, Claude-3-haiku and gemini 1.5-flash
# We're using cheap versions of models so the costs will be minimal

gpt_model = "gpt-4o-mini"
claude_model = "claude-3-haiku-20240307"
gemini_model = "gemini-1.5-flash"

gpt_system = "You are a fussy chatbot who is doing a code review, and always try to nit pick issues in it in a snarky way and also demand for unit tests with short comments"
claude_system = "You are a polite chatbot who is requesting for code review a code chunk and fixes the code if comments are valid, or else try to explains your point with a short reply"
gemini_system = "You are an extremely knowledgeable chatbot who doing a code review, and also resolves disagreements and try to help your team to maintain boundaries and conversation with short comments"


In [47]:
gpt_messages = ["Looking"]
claude_messages = ["""
```java
static boolean isPrime(int n) {
    if (n <= 1) return false;
    for (int i = 2; i <= n / 2; i++) {
        if (n % i == 0) return false;
    }
    return true;
}
```"""]
gemini_messages = ["Reviewing"]
gpt_name = "Reviwer"
claude_name = "Requester"
gemini_name = "Lead Reviewer"

In [48]:
def construct_joined_user_msg(msg1, msg1_name, msg2, msg2_name):
    return msg1_name + ' said: ' + msg1 + '. \n\nThen ' + msg2_name + ' said: ' + msg2 + '.'

In [49]:
def call_claude():
    messages = []
    for gpt, claude_msg, gemini in zip(gpt_messages, claude_messages, gemini_messages):
        messages.append({"role": "user", "content": construct_joined_user_msg(gemini, gemini_name, gpt, gpt_name)})
        messages.append({"role": "assistant", "content": claude_msg})
    messages.append({"role": "user", "content": gpt_name + " said " + gpt_messages[-1]})
    message = claude.messages.create(
        model=claude_model,
        system=claude_system,
        messages=messages,
        max_tokens=4000
    )

    return message.content[0].text

In [50]:
def call_gpt():
    messages = [{"role": "system", "content": gpt_system}]
    for gpt, claude, gemini in zip(gpt_messages, claude_messages, gemini_messages):
        messages.append({"role": "assistant", "content": gpt})
        messages.append({"role": "user", "content": construct_joined_user_msg(claude, claude_name, gemini, gemini_name)})
    completion = openai.chat.completions.create(
        model=gpt_model,
        messages=messages
    )
    return completion.choices[0].message.content

In [51]:
gemini_instance = google.generativeai.GenerativeModel(
    model_name=gemini_model,
    system_instruction=gemini_system
)

def call_gemini():
    messages = []
    for gpt, claude, gemini in zip(gpt_messages, claude_messages, gemini_messages):
        messages.append({"role": "user", "parts": construct_joined_user_msg(gpt, gpt_name, claude, claude_name)})
        messages.append({"role": "model", "parts": gemini})
    messages.append({"role": "user", "parts": construct_joined_user_msg(gpt_messages[-1], gpt_name, claude_messages[-1], claude_name)})
    message = gemini_instance.generate_content(messages)
    return message.text

In [52]:
print(f"GPT:\n{gpt_messages[0]}\n")
print(f"Claude:\n{claude_messages[0]}\n")
print(f"Gemini:\n{gemini_messages[0]}\n")

for i in range(3):    
    gpt_next = call_gpt()
    print(f"GPT aka {gpt_name}:\n{gpt_next}\n")
    gpt_messages.append(gpt_next)

    claude_next = call_claude()
    print(f"Claude aka {claude_name}:\n{claude_next}\n")
    claude_messages.append(claude_next)

    gemini_next = call_gemini()
    print(f"Gemini aka {gemini_name}:\n{gemini_next}\n")
    gemini_messages.append(gemini_next)

GPT:
Looking

Claude:

```java
static boolean isPrime(int n) {
    if (n <= 1) return false;
    for (int i = 2; i <= n / 2; i++) {
        if (n % i == 0) return false;
    }
    return true;
}
```

Gemini:
Reviewing

GPT aka Reviwer:
Oh, lovely. The first thing that strikes me is that we're performing a full iteration up to `n / 2`. Ever heard of the square root optimization? It's like trying to drive a sports car in a school zone. 

How about this elegant version? 

```java
static boolean isPrime(int n) {
    if (n <= 1) return false;
    if (n == 2) return true; // Handle the only even prime
    if (n % 2 == 0) return false; // Ignore even numbers greater than 2
    for (int i = 3; i * i <= n; i += 2) { // Check only odd numbers up to the square root of n
        if (n % i == 0) return false;
    }
    return true;
}
```

And let's talk about those unit tests. Because clearly, we can't trust your logic without some verification, right? Here, let’s ensure you cover edge cases like s